In [1]:
!pip install ktrain

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
from ktrain.text.zsl import ZeroShotClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Find the first category from the prpbability

In [4]:
def find_max_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        max_category = max(category_scores, key=lambda x: x[1])[0]
        max_categories.append(max_category)
    return max_categories

# Find the maximum first 2 categories fro mthe probability

In [5]:
def find_max_2_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        # Sort the categories based on their values in descending order
        sorted_categories = sorted(category_scores, key=lambda x: x[1], reverse=True)
        # Select the top two categories
        top_categories = [category[0] for category in sorted_categories[:2]]
        max_categories.append(top_categories)
    return max_categories

## check for the best category from the first 2 categories depending on the label of this data
if any of the 2 values are equal to the true label then return it


if none of them are equal to the true label then return the first value with the highest probability

In [6]:
def return_best_category_from_2_categories(list1, list2):
  result = []
  for item1, item2 in zip(list1, list2):
      found = False
      for value in item2:
          if value in item1:
              result.append(value)
              found = True
              break
      if not found:
          result.append(item2[0])
  return result


In [7]:
def compute_classification_report(y_true, y_pred):

    report = classification_report(y_true, y_pred)
    print(report)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [8]:
def find_max_value(category_scores):
    max_value = max(category_scores, key=lambda x: x[1])[1]
    return max_value

In [9]:
train = pd.read_csv("https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/TOPICS/DATA/news_data%206%20categories/train.csv")
test = pd.read_csv("https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/TOPICS/DATA/news_data%206%20categories/test.csv")

In [10]:
test

,Title,Excerpt,Category
0,Nigeria: Cross River Governor Ayade Dumps PDP ...,"Cross River State Governor, Ben Ayade has defe...",politics
1,Electoral Bill: Nigeria Senate Mobilises to Ov...,Nigerians have not heard the last of President...,politics
2,Nigeria: Central Bank Increases Forex Allocati...,The Central Bank of Nigeria (CBN) on Sunday an...,business
3,Nigeria to Set Up Fund to Cover 83m Poor Citiz...,President Muhammadu Buhari has signed into law...,health
4,World Bank Revises Sub-Saharan Africa’s Growth...,The World Bank has revised its growth forecast...,business
...,...,...,...
823,Nigeria’s Central Bank Introduces ‘Naira 4 Dol...,The Central Bank of Nigeria (CBN) has introduc...,business
824,Super Eagles to Host Leone Stars in AFCON 2023...,Following Tuesday night’s draw for the qualifi...,sports
825,Frustration in Nigeria Breeding Calls for Sece...,"Ekiti State Governor, Dr. Kayode Fayemi on Tue...",politics
826,Coca-Cola European Buys Australian Bottler for...,Coca-Cola European Partners Plc has agreed to ...,business


In [11]:
train.Category.value_counts()

business         1268
sports           1124
politics         1073
health            550
entertainment     382
tech              289
Name: Category, dtype: int64

In [12]:
df2 = pd.read_csv("https://github.com/mhjabreel/CharCnn_Keras/raw/master/data/ag_news_csv/train.csv")
df2.columns = ["label" , "title" , "content"]
df2['label'] = df2['label'].map({1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"})
df2

,label,title,content
0,Business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,Business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,Business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,Business,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,Business,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
...,...,...,...
119994,World,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119995,Sports,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119996,Sports,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119997,Sports,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [13]:
zsl = ZeroShotClassifier()
# labels=['business', 'sports', 'politics', 'health' , "entertainment" , "tech"]

In [14]:
doc = df2["content"].iloc[:100].values
val = df2["label"].iloc[:100].values
trn = train["Excerpt"].iloc[:100].values
y_trn = train["Category"].iloc[:100].values
tst = test["Excerpt"].values
y_tst = test["Category"].values

In [15]:
%%time
df2_prb = zsl.predict(doc, labels=["World","Sports","Business","Sci/Tech"], include_labels=True)

/usr/local/lib/python3.10/dist-packages/ktrain/text/zsl/core.py:101: UserWarning: TIP: Try increasing batch_size to speedup ZeroShotClassifier predictions
  warnings.warn(


CPU times: user 3.95 s, sys: 63 ms, total: 4.02 s
Wall time: 6.46 s


In [16]:
%%time
y_trn_prb = zsl.predict(trn, labels=['business', 'sports', 'politics', 'health' , "entertainment" , "tech"], include_labels=True ,batch_size = 16)

CPU times: user 4.38 s, sys: 24.5 ms, total: 4.41 s
Wall time: 4.95 s


In [17]:
%%time
y_tst_prb = zsl.predict(tst, labels=['business', 'sports', 'politics', 'health' , "entertainment" , "tech"], include_labels=True ,batch_size = 16)

CPU times: user 39 s, sys: 142 ms, total: 39.2 s
Wall time: 39 s


In [18]:
df2_pred = find_max_categories_for_lists(df2_prb)
y_trn_pred = find_max_categories_for_lists(y_trn_prb)
y_tst_pred = find_max_categories_for_lists(y_tst_prb)

In [19]:
compute_classification_report(val , df2_pred)

              precision    recall  f1-score   support

    Business       0.93      0.70      0.80        77
    Sci/Tech       0.76      0.57      0.65        23
      Sports       0.00      0.00      0.00         0
       World       0.00      0.00      0.00         0

    accuracy                           0.67       100
   macro avg       0.42      0.32      0.36       100
weighted avg       0.89      0.67      0.77       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.67

In [20]:
compute_classification_report(y_trn , y_trn_pred)

               precision    recall  f1-score   support

     business       0.76      0.87      0.81        30
entertainment       1.00      0.67      0.80         9
       health       0.62      0.71      0.67         7
     politics       0.91      0.81      0.86        26
       sports       1.00      0.86      0.93        22
         tech       0.50      0.83      0.62         6

     accuracy                           0.82       100
    macro avg       0.80      0.79      0.78       100
 weighted avg       0.85      0.82      0.83       100



0.82

In [21]:
compute_classification_report(y_tst , y_tst_pred)

               precision    recall  f1-score   support

     business       0.77      0.83      0.80       224
entertainment       0.95      0.81      0.87        67
       health       0.86      0.82      0.84        97
     politics       0.89      0.89      0.89       190
       sports       0.98      0.84      0.91       199
         tech       0.46      0.67      0.54        51

     accuracy                           0.84       828
    macro avg       0.82      0.81      0.81       828
 weighted avg       0.85      0.84      0.84       828



0.8357487922705314

# After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [22]:
df2_pred = find_max_2_categories_for_lists(df2_prb)
y_trn_pred = find_max_2_categories_for_lists(y_trn_prb)
y_tst_pred = find_max_2_categories_for_lists(y_tst_prb)

In [23]:
df2_pred = return_best_category_from_2_categories(val , df2_pred)
y_trn_pred = return_best_category_from_2_categories(y_trn , y_trn_pred)
y_tst_pred = return_best_category_from_2_categories(y_tst , y_tst_pred)

In [24]:
compute_classification_report(val , df2_pred)

              precision    recall  f1-score   support

    Business       0.97      0.87      0.92        77
    Sci/Tech       0.86      0.78      0.82        23
      Sports       0.00      0.00      0.00         0
       World       0.00      0.00      0.00         0

    accuracy                           0.85       100
   macro avg       0.46      0.41      0.43       100
weighted avg       0.94      0.85      0.89       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.85

In [25]:
compute_classification_report(y_trn , y_trn_pred)

               precision    recall  f1-score   support

     business       0.88      0.97      0.92        30
entertainment       1.00      0.78      0.88         9
       health       0.83      0.71      0.77         7
     politics       0.93      0.96      0.94        26
       sports       1.00      0.95      0.98        22
         tech       1.00      1.00      1.00         6

     accuracy                           0.93       100
    macro avg       0.94      0.90      0.91       100
 weighted avg       0.93      0.93      0.93       100



0.93

In [26]:
compute_classification_report(y_tst , y_tst_pred)

               precision    recall  f1-score   support

     business       0.94      0.99      0.97       224
entertainment       0.98      0.93      0.95        67
       health       0.97      0.89      0.92        97
     politics       0.98      0.97      0.98       190
       sports       0.99      0.98      0.99       199
         tech       0.89      0.96      0.92        51

     accuracy                           0.96       828
    macro avg       0.96      0.95      0.96       828
 weighted avg       0.97      0.96      0.96       828



0.964975845410628